## PREP AND BASELINE 

### langchain → our orchestration framework

### langchain-community → integrations like FAISS

### langchain-groq → to call Groq’s free LLaMA 3 models

### faiss-cpu → local vector DB

### sentence-transformers → local embeddings

### python-dotenv → load API keys from .env

In [2]:
!pip install langchain langchain-community langchain-groq faiss-cpu sentence-transformers
!pip install python-dotenv

   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   ----- ---------------------------------- 2.4/18.2 MB 12.2 MB/s eta 0:00:02
   ----------- ---------------------------- 5.2/18.2 MB 12.8 MB/s eta 0:00:02
   ------------------ --------------------- 8.4/18.2 MB 14.5 MB/s eta 0:00:01
   ----------------------------- ---------- 13.4/18.2 MB 16.4 MB/s eta 0:00:01
   ---------------------------------------  17.8/18.2 MB 17.6 MB/s eta 0:00:01
   ---------------------------------------- 18.2/18.2 MB 16.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
    --------------------------------------- 4.2/241.3 MB 19.4 MB/s eta 0:00:13
   - -------------------------------------- 7.9/241.3 MB 18.7 MB/s eta 0:00:13
   - -------------------------------------- 11.0/241.3 MB 20.2 MB/s eta 0:00:12
   -- ------------------------------------- 12.8/241.3 MB 15.2 MB/s eta 0:00:16
   -- ------------------------------------- 16.5/241.3 MB 15.8 MB/s et

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.5.23 requires tokenizers<=0.20.3,>=0.13.2, but you have tokenizers 0.21.4 which is incompatible.


### Create a .env file in the same folder as your notebook. Grab your Groq API Key from : https://console.groq.com/keys

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

if not GROQ_API_KEY:
    raise ValueError("⚠️ Please set GROQ_API_KEY in your .env file")

### For now, we’ll hardcode small policies, FAQs, and past conversation examples.

In [4]:
knowledge_base = [
    {
        "source": "policy",
        "text": "Return Policy: Customers can return products within 30 days of purchase with the original receipt."
    },
    {
        "source": "faq",
        "text": "FAQ: Our customer support is available 24/7 via chat and email."
    },
    {
        "source": "history",
        "text": "Customer: I ordered a laptop last week, when will it arrive? Agent: Your laptop is scheduled to arrive by Friday."
    }
]

### Create embeddings & store in FAISS

In [5]:
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import FAISS

# Create local embeddings model
embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Extract texts
texts = [doc["text"] for doc in knowledge_base]

# Create FAISS vector store
vectorstore = FAISS.from_texts(texts, embedding_model)

print("✅ Vector store created with", len(texts), "documents")

C:\Users\saket.khopkar\AppData\Local\Temp\ipykernel_11676\1996417352.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\saket.khopkar\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\saket.khopkar\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Vector store created with 3 documents


### Set up free LLM via Groq

In [6]:
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA

# LLaMA 3 free via Groq
llm = ChatGroq(
    groq_api_key=GROQ_API_KEY,
    model_name="llama3-8b-8192",
    temperature=0.1
)

# Create retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

# Build simple RAG pipeline
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"
)

### TESTING

In [7]:
query = "What is your return policy?"
response = qa_chain.run(query)
print("Q:", query)
print("A:", response)

query = "When will my laptop arrive?"
response = qa_chain.run(query)
print("\nQ:", query)
print("A:", response)

C:\Users\saket.khopkar\AppData\Local\Temp\ipykernel_11676\4052937287.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)


Q: What is your return policy?
A: According to our Return Policy, customers can return products within 30 days of purchase with the original receipt.

Q: When will my laptop arrive?
A: According to the information I have, your laptop is scheduled to arrive by Friday.


In [8]:
query = "How can I contact customer support?"
response = qa_chain.run(query)
print("Q:", query)
print("A:", response)

Q: How can I contact customer support?
A: According to our FAQ, you can contact our customer support 24/7 via chat and email.


In [9]:
query = "Do you have 24/7 support?"
response = qa_chain.run(query)
print("Q:", query)
print("A:", response)

Q: Do you have 24/7 support?
A: Yes, our customer support is available 24/7 via chat and email.


In [10]:
query = "Can I return an item after a month?"
response = qa_chain.run(query)
print("Q:", query)
print("A:", response)

Q: Can I return an item after a month?
A: No, according to our Return Policy, you can return an item within 30 days of purchase with the original receipt. If it's been more than 30 days, you would not be able to return the item.


In [11]:
query = "I bought an item and it broke. Can I return it?"
response = qa_chain.run(query)
print("Q:", query)
print("A:", response)

Q: I bought an item and it broke. Can I return it?
A: Yes, you can return the item. According to our Return Policy, customers can return products within 30 days of purchase with the original receipt. Since you purchased the item recently, you should be able to return it.


In [12]:
query = "If my order is delayed, what should I do?"
response = qa_chain.run(query)
print("Q:", query)
print("A:", response)

Q: If my order is delayed, what should I do?
A: If your order is delayed, I would recommend contacting the agent who provided you with the estimated delivery date. They may be able to provide you with an updated estimated delivery date or offer alternative solutions. Additionally, you can also check the tracking information for your order to see if there are any updates on the delivery status. If you have any concerns or issues with your order, you can also reach out to the customer service team for assistance.
